Installation Instructions are on 
http://www.picnet.com.au/blogs/guido/post/2016/09/22/xgboost-windows-x64-binaries-for-download/

In [1]:
import xgboost as xg

In [2]:
xg.VERSION_FILE

'C:\\Users\\Romi\\Anaconda3\\envs\\Default\\lib\\site-packages\\xgboost-0.6-py3.6.egg\\xgboost\\VERSION'

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

get_ipython().magic('matplotlib inline')
sns.set(style='white', font_scale=0.9)

#XGBoost imports
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection , metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search

In [1]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#We will also create a total dataset for dealing with missing values
df_total = df_train.append(df_test)


NameError: name 'pd' is not defined

In [ ]:
df_train.head()

In [2]:
df_train.info()

NameError: name 'df_train' is not defined

In [ ]:
df_train.groupby('Pclass')['Survived'].mean()

In [ ]:
sns.countplot(x='Pclass', data=df_train, hue='Survived', palette='deep')
plt.show()

In [ ]:
df_train['Name'].head(10)

In [ ]:
df_train['Title'] = df_train['Name'].str.split(".").str[0]
df_train['Title'] = df_train['Title'].str.split(" ").str[-1]

df_test['Title'] = df_test['Name'].str.split(".").str[0]
df_test['Title'] = df_test['Title'].str.split(" ").str[-1]

#Drop name from the dataset
df_train.drop('Name', axis=1, inplace=True)
df_test.drop('Name', axis=1, inplace=True)

In [ ]:
df_train['Title'].value_counts()

In [ ]:
def updateTitle(s):
    if s in ['Mlle', 'Ms', 'Mme']:
        return 'Miss'
    elif s in ['Mr', 'Miss', 'Mrs', 'Master']:
        return s
    else:
        return 'Other'

df_train['Title'] = df_train['Title'].apply(lambda x: updateTitle(x))

df_test['Title'] = df_test['Title'].apply(lambda x: updateTitle(x))

In [ ]:
df_train.groupby('Title')['Survived'].mean()

In [ ]:
nNoSurvived = len(df_train[df_train['Survived'] == 0])
nSurvived = len(df_train[df_train['Survived'] == 1])

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(9, 3))

sns.barplot(x='Title', y='PassengerId', data=df_train, 
            estimator=lambda x: len(x) / (nSurvived + nNoSurvived) * 100,
            ax=ax1, order=sorted(df_train['Title'].unique()), palette='deep')
sns.barplot(x='Title', y='PassengerId', data=df_train[df_train['Survived'] == 0], 
            estimator=lambda x: len(x) / nNoSurvived * 100,
            ax=ax2, order=sorted(df_train['Title'].unique()), palette='deep')
sns.barplot(x='Title', y='PassengerId', data=df_train[df_train['Survived'] == 1], 
            estimator=lambda x: len(x) / nSurvived * 100,
            ax=ax3, order=sorted(df_train['Title'].unique()), palette='deep')

ax1.set_title('Overall')
ax2.set_title('Not Survived')
ax3.set_title('Survived')
ax1.set_ylabel('Percentage')
ax2.set_ylabel('')
ax3.set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
df_train.groupby('Sex')['Survived'].mean()

In [ ]:
df_train['Age'] = df_train['Age'].fillna(df_total['Age'].mean())
df_test['Age'] = df_test['Age'].fillna(df_total['Age'].mean())

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 5))

sns.distplot(df_train[df_train['Survived'] == 0]['Age'].astype(int), bins=40, ax=ax1, kde=True)
sns.distplot(df_train[df_train['Survived'] == 1]['Age'].astype(int), bins=40, ax=ax2, kde=True)

ax1.set_title('Not Survived')
ax2.set_title('Survived')

ax1.set_xbound(lower=0, upper=100)
ax2.set_xbound(lower=0, upper=100)

plt.tight_layout()
plt.show()

In [ ]:
df_train['Parch'].value_counts()

In [ ]:
df_train['SibSp'].value_counts()


In [ ]:
df_train.groupby('SibSp')['Survived'].mean()

In [ ]:
df_train.groupby('Parch')['Survived'].mean()

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] 
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] 

df_train.drop('SibSp', axis=1, inplace=True)
df_test.drop('SibSp', axis=1, inplace=True)

df_train.drop('Parch', axis=1, inplace=True)
df_test.drop('Parch', axis=1, inplace=True)

In [ ]:
df_train['FamilySize'] = df_train['FamilySize'].apply(lambda x: 4 if x > 4 else x)
df_test['FamilySize'] = df_test['FamilySize'].apply(lambda x: 4 if x > 4 else x)

In [ ]:
df_train.groupby('FamilySize')['Survived'].mean()

In [ ]:
df_train['Ticket'] = df_train['Ticket'].str[0:1]
df_test['Ticket'] = df_train['Ticket'].str[0:1]

In [ ]:
df_train.groupby('Ticket')['Survived'].mean()

In [ ]:
df_train['Fare'] = df_train['Fare'].fillna(df_total['Fare'].mean())
df_test['Fare'] = df_test['Fare'].fillna(df_total['Fare'].mean())

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 5))

sns.distplot(df_train[df_train['Survived'] == 0]['Fare'].astype(int), bins=30, ax=ax1, kde=True)
sns.distplot(df_train[df_train['Survived'] == 1]['Fare'].astype(int), bins=30, ax=ax2, kde=True)

ax1.set_title('Not Survived')
ax2.set_title('Survived')

ax1.set_xbound(lower=0)
ax2.set_xbound(lower=0)

plt.tight_layout()
plt.show()

In [ ]:
df_train.groupby('Survived')['Fare'].describe()

In [ ]:
df_train['Cabin'] = df_train['Cabin'].str[0]
df_train['Cabin'] = df_train['Cabin'].fillna('Unknown')

df_test['Cabin'] = df_test['Cabin'].str[0]
df_test['Cabin'] = df_test['Cabin'].fillna('Unknown'

In [ ]:
df_train['Cabin'].value_counts()


In [ ]:
df_train.groupby('Cabin')['Survived'].mean()

In [ ]:
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_test['Embarked'] = df_test['Embarked'].fillna('S')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train_comb = df_train
df_train_comb['TrainTest'] = 'Train'
df_test_comb = df_test
df_test_comb['TrainTest'] = 'Test'

df_total = df_train_comb.append(df_test_comb)

In [ ]:
df_total = pd.get_dummies(df_total, drop_first=True)

In [ ]:
X_train = df_total[df_total['TrainTest_Train'] == 1].drop(['Survived', 'PassengerId'], axis=1)
y_train = df_total[df_total['TrainTest_Train'] == 1]['Survived']

X_test = df_total[df_total['TrainTest_Train'] == 0].drop(['Survived', 'PassengerId'], axis=1)

In [ ]:
param_grid1 = {'max_depth':range(3,10,1),
               'min_child_weight':range(1,6,1)}

model = XGBClassifier(learning_rate =0.1, 
                      n_estimators=140, 
                      max_depth=5,
                      min_child_weight=1, 
                      gamma=0, 
                      subsample=0.8, 
                      colsample_bytree=0.8,
                      objective= 'binary:logistic', 
                      nthread=4, 
                      scale_pos_weight=1, 
                      seed=29)

grid1 = GridSearchCV(estimator=model, param_grid=param_grid1, scoring='roc_auc', n_jobs=-1, verbose=1)
grid1.fit(X_train, y_train)

print(grid1.best_score_)
print(grid1.best_params_)

In [ ]:
param_grid2 = {'gamma':[i/10.0 for i in range(0,7)]}

model = XGBClassifier(learning_rate =0.1, 
                      n_estimators=140, 
                      max_depth=4,
                      min_child_weight=5, 
                      gamma=0, 
                      subsample=0.8, 
                      colsample_bytree=0.8,
                      objective= 'binary:logistic', 
                      nthread=4, 
                      scale_pos_weight=1, 
                      seed=29)

grid2 = GridSearchCV(estimator=model, param_grid=param_grid2, scoring='roc_auc', n_jobs=-1, verbose=1)
grid2.fit(X_train, y_train)

print(grid2.best_score_)
print(grid2.best_params_)

In [ ]:
param_grid3 = {'subsample':[i/100.0 for i in range(60,100,5)],
               'colsample_bytree':[i/100.0 for i in range(60,100,5)]}

model = XGBClassifier(learning_rate =0.1, 
                      n_estimators=140, 
                      max_depth=4,
                      min_child_weight=5, 
                      gamma=0.0, 
                      subsample=0.8, 
                      colsample_bytree=0.8,
                      objective= 'binary:logistic', 
                      nthread=4, 
                      scale_pos_weight=1, 
                      seed=29)

grid3 = GridSearchCV(estimator=model, param_grid=param_grid3, scoring='roc_auc', n_jobs=-1, verbose=1)
grid3.fit(X_train, y_train)

print(grid3.best_score_)
print(grid3.best_params_)

In [ ]:
param_grid4 = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}

model = XGBClassifier(learning_rate =0.1, 
                      n_estimators=140, 
                      max_depth=4,
                      min_child_weight=5, 
                      gamma=0, 
                      subsample=0.8, 
                      colsample_bytree=0.95,
                      objective= 'binary:logistic', 
                      nthread=4, 
                      scale_pos_weight=1, 
                      seed=29)

grid4 = GridSearchCV(estimator=model, param_grid=param_grid4, scoring='roc_auc', n_jobs=-1, verbose=1)
grid4.fit(X_train, y_train)

print(grid4.best_score_)
print(grid4.best_params_)

In [ ]:
classifier = XGBClassifier(learning_rate =0.01, 
                      n_estimators=5000, 
                      max_depth=4,
                      min_child_weight=5, 
                      gamma=0, 
                      subsample=0.8, 
                      colsample_bytree=0.95,
                      reg_alpha=1e-05,
                      objective= 'binary:logistic', 
                      nthread=4, 
                      scale_pos_weight=1, 
                      seed=29)

classifier.fit(X_train, y_train)

In [ ]:
feature_import = pd.DataFrame.from_dict(classifier.booster().get_fscore(), orient='index')
feature_import.columns = ['values']
feature_import.sort_values(['values'], ascending=False, inplace=True)
feature_import.reset_index(level=0, inplace=True)
sns.barplot(x='index', y='values', data=feature_import, palette='deep')
plt.xticks(rotation=90)
plt.show()

In [ ]:
test_pred = classifier.predict(X_test)